In [1]:
!nvidia-smi

Wed Nov 17 13:35:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
| 60%   52C    P8    48W / 370W |     48MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
| 76%   

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
import os
from tableone import TableOne, load_dataset
import pandas as pd

 
print(tf.__version__)

warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[1], device_type='GPU')

2.5.0


In [3]:
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
df_icu = pd.read_csv('Data/ICU_subject.csv')
df_icu.fillna(0, inplace=True)
df_icu

,icu_subject,ventilation_status,StudyID,COPD
0,10000032,Oxygen,53911762,1
1,10000032,Oxygen,56699142,1
2,10000032,Oxygen,53189527,1
3,10000032,Oxygen,50414267,1
4,10000980,0,54577367,1
...,...,...,...,...
127148,19999287,Oxygen,52519175,1
127149,19999287,Oxygen,50000173,1
127150,19999287,Oxygen,50574077,1
127151,19999287,Oxygen,58938059,1


In [5]:
df_ed = pd.read_csv('Data/ED_subject.csv')
df_ed.fillna(0, inplace=True)
df_ed

,ed_subject,ventilation_status,StudyID,COPD
0,10000032,0.0,53911762,0
1,10000032,0.0,56699142,0
2,10000032,0.0,53189527,0
3,10000032,0.0,50414267,0
4,10000764,0.0,57375967,0
...,...,...,...,...
227296,19999442,0.0,58497551,0
227297,19999733,0.0,57132437,0
227298,19999987,0.0,58621812,0
227299,19999987,0.0,58971208,0


In [6]:
unique_subject_id_icu = list(set(df_icu['icu_subject']))
unique_subject_id_ed = list(set(df_ed['ed_subject']))

In [7]:
demographic = pd.read_csv('Data/demographic.csv')
import random
from statistics import mode, StatisticsError

def most_common(l):
    if (len(l) == 1):
        return l[0]
    
    try:
        most_common = mode(l)
    except:
        most_common = random.choice(l)
    
    if (most_common == 3): 
        second_common_list = [x for x in l if x != most_common]
        if (len(second_common_list) > 0):
            most_common = mode(second_common_list)
    return most_common

In [8]:
from IPython import display
new_race_list = []
for i in demographic['race']:
    l = list(i)
    l = [x for x in l if x != ',']
    new_race_list.append(most_common(l))

In [9]:
demographic.drop('race',axis=1, inplace=True)
demographic['race'] = new_race_list

In [10]:
demographic.to_csv('Data/Processed_demo.csv')

In [11]:
# create a dataframe containing each patient uniquely
gender_array = []
age_array = []
races_array = []
copd_array = []
vs_array = []
miss = 0
for subject_id in unique_subject_id_icu:    
    try:
        gender = int(demographic.loc[demographic['subject_id'] == subject_id, 'gender'].values[0])
        age = int(demographic.loc[demographic['subject_id'] == subject_id, 'anchor_age'].values[0])
        race = int(demographic.loc[demographic['subject_id'] == subject_id, 'race'].values[0])
    except:
        miss += 0
        gender = np.nan
        age = np.nan
        race = np.nan
    copd_array.append(df_icu.loc[df_icu['icu_subject'] == subject_id, 'COPD'].values[0])
    vs_array.append(df_icu.loc[df_icu['icu_subject'] == subject_id, 'ventilation_status'].values[0])
    gender_array.append(gender)
    age_array.append(age)
    races_array.append(race)

unique_df_icu = pd.DataFrame({'subject_id':unique_subject_id_icu, 'gender': gender_array, 
                              'age':age_array, 'race':races_array, 
                              'COPD':copd_array, 'ventilation_status': vs_array
                             })
print(miss)
unique_df_icu

0


,subject_id,gender,age,race,COPD,ventilation_status
0,12025857,0,3,1,0,0
1,14811141,0,4,0,1,Oxygen
2,17465349,1,4,0,1,Oxygen
3,17367047,1,3,0,0,0
4,13139976,1,3,0,0,Oxygen
...,...,...,...,...,...,...
14857,10289146,1,3,3,0,Oxygen
14858,12156923,0,3,0,0,Oxygen
14859,10780669,1,3,0,0,Oxygen
14860,11141118,0,4,0,0,Oxygen


In [12]:
# create a dataframe containing each patient uniquely
gender_array = []
age_array = []
races_array = []
copd_array = []
vs_array = []
miss = 0
for subject_id in unique_subject_id_ed:
    try:
        gender = int(demographic.loc[demographic['subject_id'] == subject_id, 'gender'].values[0])
        age = int(demographic.loc[demographic['subject_id'] == subject_id, 'anchor_age'].values[0])
        race = int(demographic.loc[demographic['subject_id'] == subject_id, 'race'].values[0])
    except:
        miss += 0
        gender = np.nan
        age = np.nan
        race = np.nan
    copd_array.append(df_ed.loc[df_ed['ed_subject'] == subject_id, 'COPD'].values[0])
    vs_array.append(df_ed.loc[df_ed['ed_subject'] == subject_id, 'ventilation_status'].values[0])
    gender_array.append(gender)
    age_array.append(age)
    races_array.append(race)

unique_df_ed = pd.DataFrame({'subject_id':unique_subject_id_ed, 'gender': gender_array, 
                              'age':age_array, 'race':races_array, 
                              'COPD':copd_array, 'ventilation_status': vs_array
                             })
print(miss)
unique_df_ed

KeyboardInterrupt: 

In [ ]:
unique_df_icu = unique_df_icu.replace({'ventilation_status': {np.nan: 0, 'Oxygen': 1, 'HighFlow': 2, 'Oxygen,HighFlow': 3}})
unique_df_icu

In [ ]:
unique_df_ed = unique_df_ed.replace({'ventilation_status': {np.nan: 0, 'Oxygen': 1, 'HighFlow': 2,'Oxygen,HighFlow': 3}})
unique_df_ed

In [ ]:
df_ed = pd.DataFrame()
df_icu = pd.DataFrame()
miss = 0
filename = ['TFrecords/mimic-tf-record{i}.tfrecords'.format(i=i) for i in range(24)]
raw_dataset = tf.data.TFRecordDataset(filename)

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 
    

    subject_id = example.features.feature['subject_id'].int64_list.value[0] # change subject_id to type int
    try:
        df_ed = df_ed.append(unique_df_ed.loc[unique_df_ed['subject_id'] == subject_id], ignore_index=True)
    except:
        miss+=1

    try:
        df_icu = df_icu.append(unique_df_icu.loc[unique_df_icu['subject_id'] == subject_id], ignore_index=True)
    except:
        miss+=1


In [ ]:
df_icu = df_icu.drop_duplicates()
df_icu

In [ ]:
df_ed = df_ed.drop_duplicates()
df_ed

In [ ]:
columns = ['gender', 'age', 'race', 'COPD', 'ventilation_status']
categorical = columns
groupby = 'COPD'
mytable = TableOne(df_icu, columns=columns, categorical=categorical,
                   groupby=groupby, pval=True)

In [ ]:
mytable

In [ ]:
columns = ['gender', 'age', 'race', 'COPD', 'ventilation_status']
categorical = columns
groupby = 'COPD'
mytable = TableOne(df_ed, columns=columns, categorical=categorical,
                   groupby=groupby, pval=True)
mytable
# 0: Female, 1: Male
# 0: White, 1: Black, 2: Latino, 3: others, 4: Asian
# 0: -20, 1: 20-40, 2:40-60, 3: 60-80, 4: 80-

In [29]:
a = np.array(df_icu['subject_id'])
b = np.array(df_ed['subject_id'])
concat = np.unique(np.concatenate((a, b), axis=0))

In [30]:
train, test = train_test_split(concat, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [32]:
np.savetxt('Data/COPD_val_list.csv', val, delimiter=",")
np.savetxt('Data/COPD_test_list.csv', test, delimiter=",")
np.savetxt('Data/COPD_train_list.csv', train, delimiter=",")